In [ ]:
"""

bleaching characteristics of iGABASnFR vs 514.4409
20201124_iGABASnFR_imaging_IK experiment
50% XCite light source
14 DIV
24-well plate
taken in buffer
2 FOVs / 2 wells / 1 transfection

"""

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib import rcParams

rcParams.update({'figure.autolayout': True})

def calc_bleaching(pickle_dir, pkl):
    '''
    return pandas array of all [n_time_points x n_trials]
    '''
    
    all_exp = np.zeros((60,len(pkl))) # assuming 60 samples
    
    frame_period = 10
    for i,p in enumerate(pkl):
        d = pd.read_pickle(os.path.join(pickle_dir, p + ' bleach.pkl'))
        d_bg = pd.read_pickle(os.path.join(pickle_dir, p + ' bleach bg.pkl'))
        t_vec = d['X0'].to_numpy() * frame_period
        y_vec = d['Y0'].to_numpy()
        bg_vec = d_bg['Y0'].to_numpy()
        y_vec = y_vec - bg_vec
        all_exp[:,i] = y_vec / np.max(y_vec) # norm to 1
    
    t_vec = d['X0'].to_numpy() * frame_period
    return all_exp, t_vec

pkl_dir = r'./data/bleaching'
pickles_1 = ['1_1', '1_2', '1_3', '1_4']
pickles_4409 = ['4409_1', '4409_2', '4409_3', '4409_4']

bleach_1, t = calc_bleaching(pkl_dir, pickles_1)
bleach_1_mean = np.mean(bleach_1, axis=1)
bleach_1_std = np.std(bleach_1, axis=1)

f, ax = plt.subplots(figsize=[3,3])
plt.plot(t, bleach_1_mean, 'black')
plt.fill_between(t, bleach_1_mean+bleach_1_std, bleach_1_mean-bleach_1_std, 
                 facecolor="black", # The fill color
                 color='black',       # The outline color
                 alpha=0.2)          # Transparency of the fill)

bleach_4409, t = calc_bleaching(pkl_dir, pickles_4409)
bleach_4409_mean = np.mean(bleach_4409, axis=1)
bleach_4409_std = np.std(bleach_4409, axis=1)

plt.plot(t, bleach_4409_mean, 'red')
plt.fill_between(t, bleach_4409_mean+bleach_4409_std, bleach_4409_mean-bleach_4409_std, 
                 facecolor="red", # The fill color
                 color='red',       # The outline color
                 alpha=0.2)          # Transparency of the fill)

ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Norm. fluorescence', fontsize=12)
ax.set_ylim(0,1.2)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.legend(['iGABASnFR', 'iGABASnFR2'])
f.savefig('figs/bleaching.pdf')

# stats
print('bleaching after 10 mins:')
print('iGABASnFR lost: {:.2f}+/-{:.2f} (n={})'.format(bleach_1_mean[-1], bleach_1_std[-1], len(pickles_1)))
print('iGABASnFR2 lost: {:.2f}+/-{:.2f} (n={})'.format(bleach_4409_mean[-1], bleach_4409_std[-1], len(pickles_4409)))